In [2]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [8]:
# Load data and select columns
# path = 'C:/_data/'
path = ""  # <- 지훈씨가 쓸 때는 이걸 주석처리하고 위에 껄 쓰시면 됩니다.
ts = 1
df = pd.read_csv(path + "아파트(매매).csv")

df = df[["거래금액(만원)","본번", "부번", "단지명", "전용면적(㎡)", "계약년월","층", "건축년도"]]
df.head()

,거래금액(만원),본번,부번,단지명,전용면적(㎡),계약년월,층,건축년도
0,"60,000",655,2,개포2차현대아파트(220),77.75,200606,4,1988
1,"72,250",655,2,개포2차현대아파트(220),77.75,200610,5,1988
2,"73,500",655,2,개포2차현대아파트(220),77.75,200610,8,1988
3,"76,500",655,2,개포2차현대아파트(220),77.75,200612,5,1988
4,"88,000",658,1,개포6차우성아파트1동~8동,79.97,200605,3,1987


In [6]:
# df.drop('본번', axis=1)
# df.drop('부번', axis=1)
# df.drop('단지명', axis=1)
# df.drop('전용면적(㎡)', axis=1)
# df.drop('계약년월', axis=1)
# df.drop('건축년도', axis=1)

# 아래처럼 리스트로 전달하면 한 줄로 원하는 컬럼을 모두 드랍할 수 있습니다.
# 그리고 drop() 함수는 디폴트 값이 inplace = False 이기 때문에,
# df = df.drop('건축년도', axis=1) <- 이렇게 다시 df 에 할당 해주거나,
# df.drop('건축년도', axis=1, inplace=True) <- inplace 파라미터에 True 값을 전달해서
# df 라는 변수에 다시 재할당 하지 않도록 하는 방법이 있어요.
df.drop(['본번','부번','단지명','전용면적(㎡)','계약년월','건축년도'], axis=1, inplace=True)
df.head()

,거래금액(만원),층
0,"60,000",4
1,"72,250",5
2,"73,500",8
3,"76,500",5
4,"88,000",3


In [ ]:
# Fit label encoders to categorical columns
label_encoders = {}
for col in df.select_dtypes(include=['object']):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [ ]:
# Print label mappings for each column
for col, le in label_encoders.items():
    print(col, dict(zip(le.classes_, le.transform(le.classes_))))

In [ ]:
# Split data into train and test sets
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

In [ ]:
# Scale data using MinMaxScaler
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [ ]:
# Prepare data for LSTM model
def prepare_data(data, time_steps=1):
    X, Y = [], []
    for i in range(len(data) - time_steps):
        a = data[i:(i + time_steps), :]
        X.append(a)
        Y.append(data[i + time_steps, 0])
    return np.array(X), np.array(Y)

In [ ]:
train_X, train_Y = prepare_data(train_data)
test_X, test_Y = prepare_data(test_data)

In [ ]:
# Build LSTM model
model = Sequential()
model.add(LSTM(10, input_shape=(ts,train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

In [ ]:
# Train LSTM model
model.fit(train_X, train_Y, epochs=200, batch_size=5, verbose=2)

In [ ]:
# Evaluate LSTM model
loss, accuracy = model.evaluate(test_X, test_Y, verbose=0)
# print('mse', mse)  # 이건 loss 를 보고 싶으섰던 건가요?
print("mse:", loss)
print('Test accuracy:', accuracy)

In [ ]:
y_pred = model.predict(test_X)
r2 = r2_score(test_Y, y_pred)
print('r2 score:', r2)